In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, Latex
df = pd.read_csv('responses_scrubbed.csv', index_col=None, parse_dates=['date'])

## Number of respondents from each neighborhood
Gives us a sense of who responded.

In [2]:
df['neighborhood'].value_counts()

Harmon                 151
Upper Village          107
Old Post Road North     68
Mount Airy              55
Sunset Park             42
Albany Post Road        29
Half Moon Bay           19
Quaker Ridge             9
Cortlandt                8
Ossining                 1
Name: neighborhood, dtype: int64

## Number of respondents from each demographic
Gives us a sense of who responded.

In [3]:
df['demographic'].value_counts()

Adult with school kid       194
Adult without school kid    165
Senior                      128
College kid                   5
High School kid               3
Middle School kid             1
Name: demographic, dtype: int64

## Percent respondents of each demographic from each neighborhood
Each column adds up to 100%.

In [4]:
df2 = df.groupby(['demographic', 'neighborhood']).size().unstack(fill_value=0).apply(lambda x: 100 * x / x.sum()) #.astype(int)
df2.round(0).astype(int) # round to int

neighborhood,Albany Post Road,Cortlandt,Half Moon Bay,Harmon,Mount Airy,Old Post Road North,Ossining,Quaker Ridge,Sunset Park,Upper Village
demographic,,,,,,,,,,
Adult with school kid,21,25,5,40,47,46,0,44,43,42
Adult without school kid,28,50,47,27,29,36,100,33,40,34
College kid,0,0,0,2,0,1,0,0,2,0
High School kid,0,0,0,1,2,0,0,0,0,0
Middle School kid,0,0,0,1,0,0,0,0,0,0
Senior,52,25,47,29,22,16,0,22,14,25


## Set up some useful functions for talllying responses

In [5]:
# splits the comma-separated values in the column into a pandas Series
def split_by_comma(group, column_title):
    try:
        value_list = group[column_title].str.split(',')
        return pd.Series(value_list.values[0], name=column_title)
    except:
        return []

# breaks up the indicated column representing checkbox type data into multiple columns
def get_answer_options(column_title, df, series_name):
    # group all respondents who gave the same child_drive_reason
    ser = df[pd.notnull(df[column_title])].groupby(level=0).apply(split_by_comma, column_title=column_title)
    try:
        # remove extra whitespace, if any responses exist
        ser = ser.map(lambda x: str(x).strip()) #strip whitespace from answers
    except:
        # there were no results in the series
        pass
    
    dum = pd.get_dummies(ser) # split each respondent's answers into separate columns
    
    # sum columns
    col_list = list(dum.columns) 
    dum.loc[series_name] = dum[col_list].sum(axis=0)
    
    # sort and return only the top 10
    return dum.loc[series_name].sort_values(ascending=False)[0:10].sort_index()

# entry point
def get_responses(column_title, df, demographic='Anyone', neighborhood='Anywhere', as_percent=True, series_name='overall'):
    # how many people gave reasons of any kind?
    num_respondents = df[pd.notnull(df[column_title])].shape[0]
    
    # break up the column of interest
    results = get_answer_options(column_title, df, series_name)
    
    # calculate percentages, if requested
    if as_percent:
        results = results.apply(lambda x: int(100 * x / num_respondents))

    return results

# builds a set of columns representing each demographic's 
def break_down_by(column_title, breakdown_column='demographic'):
    
    # get the overall reasons people gave for each
    responses = get_responses(column_title, df, series_name='overall')
    # convert to dataframe
    df2 = responses.to_frame()
    
    # add on columns for each demographic to the dataframe
    for breakdown_index, v in df[breakdown_column].value_counts().items():
        df3 = df[df[breakdown_column] == breakdown_index ]
        try:
            # get the reasons this specific demographic gave for this column
            responses = get_responses(column_title, df3, breakdown_index, series_name=breakdown_index)
            #df2 = pd.concat([df2, responses.to_frame()], axis=1)
            df2 = df2.join(responses.to_frame(), how='outer')
            #print(df2)
        except:
            # there were no responses from this demographic
            continue
            
    return df2


# Analyze responses

In [38]:
# a list of multiple-choice columns we'd like to analyze
fields = [ 'schools', 'child_drive_reason', 'child_no_walk_reason', 'child_no_bike_reason', 'commuter_distance', 'no_walk_reason', 'no_bike_reason', 'drive_reason' ]

# builds a set of columns representing each demographic's 
def break_down_by(column_title, breakdown_column='demographic'):
    
    # get the overall reasons people gave for each
    responses = get_responses(column_title, df, series_name='overall')
    # convert to dataframe
    df2 = responses.to_frame()
    
    # add on columns for each demographic to the dataframe
    for breakdown_index, v in df[breakdown_column].value_counts().items():
        df3 = df[df[breakdown_column] == breakdown_index ]
        try:
            # get the reasons this specific demographic gave for this column
            responses = get_responses(column_title, df3, breakdown_index, series_name=breakdown_index)
            df2 = pd.concat([df2, responses.to_frame()], axis=1)
            #print(df2)
        except:
            # there were no responses from this demographic
            continue
            
    return df2

        
# loop through each of the checkbox-type questions
for column_title in fields:
    
    # break this column down by demographic
    df2 = break_down_by(column_title, breakdown_column='demographic')
    
    # break this column down by demographic
    df3 = break_down_by(column_title, breakdown_column='neighborhood')
    
    # output top 10 results
    num_respondents = df[pd.notnull(df[column_title])].shape[0] # get the number of people who answered this question
    display(Markdown('## *{}* (as % of {} respondents):'.format(column_title, num_respondents).title()))

    display(Markdown('### Broken down by demographic:'.title()))
    print(df2)
    #display(Markdown('### Broken down by neighborhood:'.title()))
    #print(df3)
    

## *Schools* (As % Of 188 Respondents):

### Broken Down By Demographic:

                                   overall  Adult with school kid
CET                                     45                     45
CHHS                                    30                     30
Circle School                            1                      1
Currently pre-school but soon CET        0                      0
Happy Hearts                             1                      1
Happy Hearts Day Care                    0                      0
Homeschooled                             2                      2
PVC                                     31                     31
St. Augustine                            1                      1
The Harvey School                        0                      0


## *Child_Drive_Reason* (As % Of 189 Respondents):

### Broken Down By Demographic:

...

### Broken Down By Demographic:

                                             overall  Adult with school kid  \
0_No - I bicycle a lot                           NaN                    NaN   
1_ Lack of adequate bike lanes                   NaN                    NaN   
Croton is very hilly! It’s not easy riding.      NaN                    NaN   
Don’t know how                                   NaN                    NaN   
Fear of dangerous drivers                        NaN                    1.0   
Fear of dangerous driving                       23.0                   30.0   
Hills                                            0.0                    1.0   
I do not like to bicycle                        16.0                   17.0   
I don't have time to bicycle                    12.0                   15.0   
I don't own a Bicycle                            NaN                    NaN   
I live too far to bicycle                        3.0                    3.0   
Lack of adequate bike lanes                     22.0

## *Drive_Reason* (As % Of 476 Respondents):

### Broken Down By Demographic:

                                                    overall  \
0_My own personal preference                            NaN   
Blue Pig                                                NaN   
Convenience                                             0.0   
Distance from my airy                                   NaN   
Hills                                                   1.0   
I don't have time to walk or bicycle                   24.0   
I drop my kids off at Happy Hearts and can't bi...      NaN   
I use the car during bad weather or when I need...      NaN   
Many Overgrown sidewalks                                NaN   
My own health condition                                 4.0   
My own personal preference                             33.0   
No - I do not drive                                     7.0   
Safety concerns with bicycling                         16.0   
Safety concerns with walking                           10.0   
Time                                                   

## Who rides on the sidewalk?

### Overall

In [7]:
# do people generally bike on the sidewalk?
df['bikes_on_sidewalk'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bikes_on_sidewalk
No - I don't bicycle,49
No - I only bicycle on the road,29
Yes,23


### People who never bike

In [8]:
# do people who never bike claim to ride on the sidewalk?
df2 = df[ df['bike_freq'].isin(['Rarely or never']) ]
df2['bikes_on_sidewalk'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bikes_on_sidewalk
No - I don't bicycle,80
No - I only bicycle on the road,13
Yes,7


### People who bike at least occasionally

In [9]:
# do people who ride a bike occasionally or more ride on the sidewalk?
df2 = df[ df['bike_freq'].isin(['Every day, or almost every day', 'Once every few days', 'Once in a while']) ]
df2['bikes_on_sidewalk'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bikes_on_sidewalk
No - I don't bicycle,1
No - I only bicycle on the road,53
Yes,46


### People who bike a lot

In [10]:
# do people who bike a lot ride on the sidewalk?
df2 = df[ df['bike_freq'].isin(['Every day, or almost every day']) ]
df2['bikes_on_sidewalk'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bikes_on_sidewalk
No - I only bicycle on the road,58
Yes,42


## Who thinks drivers are safe?

### Overall

In [11]:
# do people generally think drivers are safe?
df['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()

,drivers_are_safe
1.0,9
2.0,30
3.0,47
4.0,14


### People who walk a lot

In [12]:
# do people who walk a lot think drivers are safe?
df2 = df[ df['walk_freq'].isin(['Every day, or almost every day', 'Once every few days']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,drivers_are_safe
1.0,11
2.0,32
3.0,46
4.0,10


### People who rarely walk

In [13]:
# do people who rarely walk think drivers are safe?
df2 = df[ df['walk_freq'].isin(['Rarely or never', 'Once in a while']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,drivers_are_safe
1.0,6
2.0,25
3.0,46
4.0,22


### People who never walk

In [14]:
# do people who never walk think drivers are safe?
df2 = df[ df['walk_freq'].isin(['Rarely or never']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,drivers_are_safe
1.0,10
2.0,22
3.0,42
4.0,25


### People who bike frequently

In [15]:
# do people who bike a lot think drivers are safe?
df2 = df[ df['bike_freq'].isin(['Every day, or almost every day', 'Once every few days']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()

,drivers_are_safe
1.0,13
2.0,33
3.0,47
4.0,8


## Who thinks bicyclists are safe?

### Overall

In [16]:
# do people generally think bicyclists are safe?
df['bicyclists_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bicyclists_are_safe
1.0,13
2.0,23
3.0,49
4.0,15


### People who bike a lot

In [17]:
# do people who bike a lot think bicyclists are safe
df2 = df[ df['bike_freq'].isin(['Every day, or almost every day', 'Once every few days']) ]
df2['bicyclists_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bicyclists_are_safe
1.0,5
2.0,15
3.0,57
4.0,24


### People who rarely bike

In [18]:
# do people who never bike think bicyclists are safe
df2 = df[ df['bike_freq'].isin(['Rarely or never', 'Once in a while']) ]
df2['bicyclists_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bicyclists_are_safe
1.0,15
2.0,25
3.0,46
4.0,13


In [19]:
# do people who never bike think drivers are safe?
df2 = df[ df['bike_freq'].isin(['Rarely or never', 'Once in a while']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,drivers_are_safe
1.0,9
2.0,29
3.0,47
4.0,16


In [20]:
df[df['demographic'] == 'College kid']['walk_freq'].value_counts().sort_index()

Once every few days    2
Once in a while        2
Rarely or never        1
Name: walk_freq, dtype: int64

## Additional breakdowns

In [37]:
def get_num_respondents(field, df):
    # get the number of responses in this field
    num_respondents = df[pd.notnull(df[field])].shape[0]
    return num_respondents
    
def get_breakdown(field, breakdown_field, percent=True):
    # get the number of respondents to this field
    num_respondents = get_num_respondents(field, df)

    # get the overall stats of the values in the field of interest
    df2 = df[field].value_counts()
    if percent:
        df2 = df2.apply(lambda x: int(100 * x / num_respondents))
    
    # label the column nicely 
    df2 = df2.to_frame()
    df2.columns = ['overall'] #name this 'overall', since it is not broken down

    # get the various values given in the field we want to break down the target field by
    breakdowns = df[breakdown_field].value_counts().items()

    # now break down the field by each of the values in the breakdown field
    for breakdown, value in breakdowns:        
        # get the number of responses by respondents matching this breakdown category
        df3 = df[df[breakdown_field] == breakdown][field].value_counts()
        
        # convert to percentage, if desired
        if percent:
            # get the number of respondents to this field
            num_respondents_column = get_num_respondents(field, df[df[breakdown_field] == breakdown][field].to_frame())
            
            df3 = df3.apply(lambda x: int(100 * x / num_respondents_column))

        # label the column nicely before we merge it into the other dataframe
        df3 = df3.to_frame() # convert to dataframe
        df3.columns = [breakdown]
        

        # join these two dataframes together
        df2 = df2.join(df3, how='outer')

    return df2

# fields of interest
fields = [ 'child_bus_freq', 'child_walk_freq', 'child_driven_freq', 'child_drive_freq', 'child_bike_freq', 'walk_freq', 'bike_freq', 'bikes_on_sidewalk', 'self_jog_frequency', 'commutes', 'child_self_school', 'child_self_bus_freq', 'child_self_bike_freq', 'child_self_driven_freq', 'child_self_drive_freq', 'child_self_walk_freq', 'child_self_commutes', 'child_self_has_children', 'commuter_type', 'commuter_walk_to_station_freq', 'commuter_bike_to_station_freq', 'commuter_drive_to_station_freq', 'commuter_carpool_to_station_freq', 'commuter_driven_to_station_freq', 'commuter_bus_to_station_freq', 'drivers_are_safe', 'bicyclists_are_safe', 'contact_interest' ]
# not included: ['child_drive_reason', 'child_no_walk_reason', 'child_no_bike_reason', 'commuter_distance', 'no_walk_reason', 'no_bike_reason', 'drive_reason' ]

# iterate through every field of interest
for field in fields:
    
    # get this field's breakdown by demographic and neighborhood
    df2 = get_breakdown(field, 'demographic', percent=True)
    df3 = get_breakdown(field, 'neighborhood', percent=True)
    
    # print out results
    display(Markdown('## *{}* (as % of {} respondents):'.format(field, get_num_respondents(field, df)).title()))

    display(Markdown('### by demographic:'.format(field, get_num_respondents(field, df)).title()))
    print(df2)

    display(Markdown('### by neighborhood:'.format(field, get_num_respondents(field, df)).title()))
    print(df3)


## *Child_Bus_Freq* (As % Of 193 Respondents):

### By Demographic:

                                overall  Adult with school kid  \
Every day, or almost every day       48                     48   
Rarely or never                      41                     41   
Once in a while                       5                      5   
Once every few days                   3                      3   

                                Adult without school kid  Senior  College kid  \
Every day, or almost every day                       NaN     NaN          NaN   
Rarely or never                                      NaN     NaN          NaN   
Once in a while                                      NaN     NaN          NaN   
Once every few days                                  NaN     NaN          NaN   

                                High School kid  Middle School kid  
Every day, or almost every day              NaN                NaN  
Rarely or never                             NaN                NaN  
Once in a while                             NaN         

### By Neighborhood:

                                overall  Harmon  Upper Village  \
Every day, or almost every day       48      41           46.0   
Once every few days                   3       6            NaN   
Once in a while                       5       8            6.0   
Rarely or never                      41      43           46.0   

                                Old Post Road North  Mount Airy  Sunset Park  \
Every day, or almost every day                   54          65         44.0   
Once every few days                               6           3          NaN   
Once in a while                                   3           7          NaN   
Rarely or never                                  35          23         55.0   

                                Albany Post Road  Half Moon Bay  Quaker Ridge  \
Every day, or almost every day              50.0          100.0           NaN   
Once every few days                          NaN            NaN           NaN   
Once in a while          

...

## *Contact_Interest* (As % Of 457 Respondents):

### By Demographic:

     overall  Adult with school kid  Adult without school kid  Senior  \
No        54                     52                        53      55   
Yes       45                     47                        46      44   

     College kid  High School kid  Middle School kid  
No            60            100.0              100.0  
Yes           40              NaN                NaN  


### By Neighborhood:

     overall  Harmon  Upper Village  Old Post Road North  Mount Airy  \
No        54      51             51                   54          62   
Yes       45      48             48                   45          37   

     Sunset Park  Albany Post Road  Half Moon Bay  Quaker Ridge  Cortlandt  \
No            55                59             66            33         50   
Yes           45                40             33            66         50   

     Ossining  
No      100.0  
Yes       NaN  
